# Exploring NLP Approaches for Classifying Promoter Regions in DNA Sequences

Methodology in Information and Communication Technologies


Authors: Rebeka Maneva, Konstantin Lozankoski

Necessary installations

In [ ]:
!pip install -q biopython scikit-learn tensorflow matplotlib seaborn tqdm xgboost transformers accelerate datasets pyarrow huggingface_hub
from google.colab import drive
drive.mount('/content/drive')
import numpy as np, pandas as pd, re, random, tensorflow as tf
from random import shuffle
import matplotlib.pyplot as plt, seaborn as sns
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import os
import xgboost as xgb
import torch
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve,
    precision_recall_curve, average_precision_score, classification_report
)
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
device = "cuda" if torch.cuda.is_available() else "cpu"
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HUGGINGFACE_TOKEN'))
np.random.seed(42); tf.random.set_seed(42); random.seed(42)

Search for "!!!" in the blocks to update the paths before running

# DATA

RegulonDB Promoter Dataset

https://regulondb.ccg.unam.mx/datasets

Getting the promotor sequences - generating promoter (positive) examples

In [ ]:
promoter_path = "x/promoter.tsv" # update this path !!!

# key columns
promoter_df = pd.read_csv(promoter_path, sep="\t", skiprows=33)
seq_col = [c for c in promoter_df.columns if "seq" in c.lower()][0]
pos_col = [c for c in promoter_df.columns if "pos" in c.lower()][0]

# cleaning sequence and position
promoter_df = promoter_df[[seq_col, pos_col]].dropna()
promoter_df[seq_col] = promoter_df[seq_col].apply(lambda s: re.sub(r'[^ATCG]', '', str(s).upper()))
promoter_df[pos_col] = promoter_df[pos_col].astype(int)
promoter_df = promoter_df.rename(columns={seq_col: "sequence", pos_col: "posTSS"})
promoter_df["label"] = 1

Getting sequences not containing promoters - generating nonpromoter (negative) examples

In [ ]:
gene_path = "x/gene.tsv" # update this path !!!

gene_df = pd.read_csv(gene_path, sep="\t", skiprows=29)
gene_seq_col = [c for c in gene_df.columns if "seq" in c.lower()][0]
gene_id_col = [c for c in gene_df.columns if "id" in c.lower()][0]

# cleaning up and preparing
gene_df = gene_df[[gene_id_col, gene_seq_col]].dropna(subset=[gene_seq_col])
gene_df[gene_seq_col] = gene_df[gene_seq_col].apply(lambda s: re.sub(r'[^ATCG]', '', str(s).upper()))
gene_df = gene_df.rename(columns={gene_seq_col: "sequence"})
gene_df["label"] = 0


# matching sizes to balance positives and negatives
neg_df = gene_df.sample(n=len(promoter_df), random_state=42)
balanced_df = pd.concat([promoter_df[["sequence", "label"]], neg_df[["sequence", "label"]]], ignore_index=True)
balanced_df.drop_duplicates(subset=["sequence"], inplace=True)

Getting negatives from the genome itself

In [ ]:
WINDOW = 100
MIN_DISTANCE = 1000
NEG_PER_POS = 1
FASTA_PATH = "x/sequence.fasta" # update this path !!!

random.seed(42)
np.random.seed(42)

# load genome
record = SeqIO.read(FASTA_PATH, "fasta")
genome = str(record.seq).upper()
genome_len = len(genome)

# known promoter centers (TSS positions)
known_tss = promoter_df["posTSS"].dropna().astype(int).tolist()
known_tss = [p for p in known_tss if 0 <= p < genome_len]

def extract_centered_window(genome_str, center_pos, window):
    half = window // 2
    start = center_pos - half
    end = start + window
    if start < 0 or end > len(genome_str):
        return None
    seq = genome_str[start:end]
    if set(seq) <= {"A", "T", "C", "G"}:
        return seq
    return None

def sample_genome_negatives(genome_str, known_centers, n_samples, window, min_distance):
    centers = np.array(sorted(set(known_centers)), dtype=np.int64)
    neg_seqs = []
    neg_centers = []

    def far_enough(pos):
        idx = np.searchsorted(centers, pos)
        candidates = []
        if idx > 0:
            candidates.append(abs(pos - centers[idx-1]))
        if idx < len(centers):
            candidates.append(abs(pos - centers[idx]))
        if not candidates:
            return True
        return min(candidates) >= min_distance

    attempts = 0
    max_attempts = n_samples * 200
    while len(neg_seqs) < n_samples and attempts < max_attempts:
        attempts += 1
        pos = random.randint(window//2, len(genome_str) - window//2 - 1)
        if not far_enough(pos):
            continue
        seq = extract_centered_window(genome_str, pos, window)
        if seq is None:
            continue
        neg_seqs.append(seq)
        neg_centers.append(pos)

    return neg_centers, neg_seqs

# extract positive sequences
pos_seqs = []
pos_centers = []
for tss in known_tss:
    seq = extract_centered_window(genome, tss, WINDOW)
    if seq is not None:
        pos_seqs.append(seq)
        pos_centers.append(tss)

pos_df_genome = pd.DataFrame({
    "sequence": pos_seqs,
    "label": 1,
    "center_pos": pos_centers
})

# sample negatives
n_neg = int(len(pos_df_genome) * NEG_PER_POS)
neg_centers, neg_seqs = sample_genome_negatives(
    genome, pos_df_genome["center_pos"].tolist(), n_neg, WINDOW, MIN_DISTANCE
)

neg_df_genome = pd.DataFrame({
    "sequence": neg_seqs,
    "label": 0,
    "center_pos": neg_centers
})

Getting hard negatives which are just shuffled promoters - generating fake negative examples and Slightly mutating real promoter sequences (2 % of bases) - adding diversity, combating overfitting

In [ ]:
# shuffling bases but still preserving global GC content
def gc_shuffled(seq):
    bases = list(seq)
    shuffle(bases)
    return "".join(bases)

# mutating 2% of bases
def mutate(seq, rate=0.02):
    bases = ['A','T','G','C']
    s = list(seq)
    for i in range(len(s)):
        if random.random() < rate:
            s[i] = random.choice(bases)
    return "".join(s)

# generating augmentations
fake_negatives = [gc_shuffled(s) for s in promoter_df["sequence"]]
mutated_promoters = [mutate(s, 0.02) for s in promoter_df["sequence"]]

neg_df2 = pd.DataFrame({"sequence": fake_negatives, "label": 0})
pos_df2 = pd.DataFrame({"sequence": mutated_promoters, "label": 1})

# combining all
balanced_df = pd.concat([
    balanced_df,
    neg_df2,
    pos_df2,
    neg_df_genome,
    pos_df_genome
], ignore_index=True).drop_duplicates(subset=["sequence"])

# matching lenghts
def one_hot_encode(seq, maxlen=100):
    mapping = {'A':[1,0,0,0],'T':[0,1,0,0],'G':[0,0,1,0],'C':[0,0,0,1],'N':[0,0,0,0]}
    seq = seq.upper()[:maxlen].ljust(maxlen, 'N')
    return np.array([mapping.get(base,[0,0,0,0]) for base in seq])

maxlen = 100
X = np.array([one_hot_encode(s, maxlen) for s in balanced_df.sequence])
y = balanced_df.label.values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# TRAINING AND EVALUATION

In [24]:
def make_X_y_from_df(df, window):
    X = np.array([one_hot_encode(s, window) for s in df["sequence"].values])
    y = df["label"].values.astype(int)
    return X, y

def build_cnn(window):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window, 4)),
        tf.keras.layers.Conv1D(64, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(128, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_bilstm(window):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window, 4)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_bilstm(window):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window, 4)),
        tf.keras.layers.Conv1D(64, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(128, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def eval_probs(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "ROC_AUC": roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan,
        "PR_AUC": average_precision_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan
    }


def plot_roc_pr_curves(results_probs, title_prefix=""):
    plt.figure(figsize=(6,5))
    for r in results_probs:
        fpr, tpr, _ = roc_curve(r["y_true"], r["y_prob"])
        auc = roc_auc_score(r["y_true"], r["y_prob"])
        plt.plot(fpr, tpr, label=f'{r["name"]} (AUC={auc:.3f})')
    plt.plot([0,1], [0,1], linestyle='--')
    plt.title(f"{title_prefix} ROC Curves")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()

    plt.figure(figsize=(6,5))
    for r in results_probs:
        p, rec, _ = precision_recall_curve(r["y_true"], r["y_prob"])
        ap = average_precision_score(r["y_true"], r["y_prob"])
        plt.plot(rec, p, label=f'{r["name"]} (AP={ap:.3f})')
    plt.title(f"{title_prefix} Precision–Recall Curves")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.show()

def plot_confusion(y_true, y_prob, name, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Confusion Matrix — {name} (thr={threshold})")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

All non-DNABERT models for window=100 and window=200 (Table A)

In [ ]:
WINDOW_SIZES = [100, 200]
EPOCHS = 15
BATCH_SIZE = 32

all_results_A = []
all_probs_for_plots = {}

for window in WINDOW_SIZES:
    Xw, yw = make_X_y_from_df(balanced_df, window)

    X_train, X_test, y_train, y_test = train_test_split(
        Xw, yw, test_size=0.2, stratify=yw, random_state=42
    )

    # XGBoost baseline
    X_train_flat = X_train.reshape(len(X_train), -1)
    X_test_flat  = X_test.reshape(len(X_test), -1)

    xgb_model = xgb.XGBClassifier(
        eval_metric='logloss',
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9
    )
    xgb_model.fit(X_train_flat, y_train)
    xgb_prob = xgb_model.predict_proba(X_test_flat)[:,1]

    m = eval_probs(y_test, xgb_prob, threshold=0.5)
    all_results_A.append({"Window": window, "Model":"XGBoost", **m})
    all_probs_for_plots.setdefault(window, []).append({"name":"XGBoost", "y_true":y_test, "y_prob":xgb_prob})

    # Deep models
    deep_specs = [
        ("CNN", build_cnn),
        ("BiLSTM", build_bilstm),
        ("CNN+BiLSTM", build_cnn_bilstm)
    ]

    for name, builder in deep_specs:
        model = builder(window)

        es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
        history = model.fit(
            X_train, y_train,
            validation_split=0.2,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=[es],
            verbose=0
        )

        # probs + metrics
        y_prob = model.predict(X_test, batch_size=512, verbose=0).ravel()
        metrics = eval_probs(y_test, y_prob, threshold=0.5)
        all_results_A.append({"Window": window, "Model": name, **metrics})
        all_probs_for_plots[window].append({"name":name, "y_true":y_test, "y_prob":y_prob})

        # training curves
        if window == 100:
            plt.figure(figsize=(10,4))
            plt.plot(history.history['accuracy'], label='Train acc')
            plt.plot(history.history['val_accuracy'], label='Val acc')
            plt.title(f"{name} Accuracy (window=100)")
            plt.legend()
            plt.show()

            plt.figure(figsize=(10,4))
            plt.plot(history.history['loss'], label='Train loss')
            plt.plot(history.history['val_loss'], label='Val loss')
            plt.title(f"{name} Loss (window=100)")
            plt.legend()
            plt.show()

# Table A
df_A = pd.DataFrame(all_results_A)
df_A = df_A.sort_values(["Window", "F1"], ascending=[True, False])
display(df_A)

# ROC/PR plots per window
for window in WINDOW_SIZES:
    plot_roc_pr_curves(all_probs_for_plots[window], title_prefix=f"(Window={window})")

UCI external test for window=100 and window=200 (Table B)

In [ ]:
def load_uci_promoters(path):
    sequences, labels = [], []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            first = line[0]
            if first == '+':
                y = 1
            elif first == '-':
                y = 0
            else:
                parts = line.split()
                if parts and parts[0] in ['+','-']:
                    y = 1 if parts[0] == '+' else 0
                else:
                    continue

            seq = line.split()[-1].replace('.', '').upper()
            seq = re.sub(r'[^ATCG]', '', seq)

            if len(seq) > 0:
                sequences.append(seq)
                labels.append(y)

    uci_df = pd.DataFrame({"sequence": sequences, "label": labels})
    return uci_df

uci_df = load_uci_promoters("x/uci.data")  # update this path !!!


all_results_B = []

for window in WINDOW_SIZES:
    # encoding UCI for this window
    X_uci, y_uci = make_X_y_from_df(uci_df, window)

    # rebuildng train/test on RegulonDB for this window (so models match window)
    Xw, yw = make_X_y_from_df(balanced_df, window)
    X_train, X_test, y_train, y_test = train_test_split(
        Xw, yw, test_size=0.2, stratify=yw, random_state=42
    )

    # XGBoost
    xgb_model = xgb.XGBClassifier(
        eval_metric='logloss',
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9
    )
    xgb_model.fit(X_train.reshape(len(X_train), -1), y_train)
    uci_prob = xgb_model.predict_proba(X_uci.reshape(len(X_uci), -1))[:,1]
    m = eval_probs(y_uci, uci_prob, threshold=0.5)
    all_results_B.append({"Window": window, "Model":"XGBoost", **m})

    # Deep models
    for name, builder in [("CNN", build_cnn), ("BiLSTM", build_bilstm), ("CNN+BiLSTM", build_cnn_bilstm)]:
        model = builder(window)
        es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
        model.fit(X_train, y_train, validation_split=0.2, epochs=15, batch_size=32, callbacks=[es], verbose=0)

        uci_prob = model.predict(X_uci, batch_size=512, verbose=0).ravel()
        m = eval_probs(y_uci, uci_prob, threshold=0.5)
        all_results_B.append({"Window": window, "Model": name, **m})

df_B = pd.DataFrame(all_results_B).sort_values(["Window", "F1"], ascending=[True, False])
display(df_B)

DNABERT model

In [ ]:
MODEL_ID = "zhihan1996/DNA_bert_6"

def clean_seq(seq: str) -> str:
    return re.sub(r'[^ATCG]', '', str(seq).upper())

def normalize_len(seq: str, window: int) -> str:
    seq = clean_seq(seq)
    if len(seq) >= window:
        return seq[:window]
    pad = "".join(random.choice("ATCG") for _ in range(window - len(seq)))
    return seq + pad

def seq_to_kmers(seq: str, k: int = 6) -> str:
    seq = clean_seq(seq)
    if len(seq) < k:
        return seq
    return " ".join(seq[i:i+k] for i in range(len(seq) - k + 1))

def safe_auc(y_true, y_score):
    y_true = np.asarray(y_true)
    if len(np.unique(y_true)) < 2:
        return float("nan")
    return roc_auc_score(y_true, y_score)

def safe_prauc(y_true, y_score):
    y_true = np.asarray(y_true)
    if len(np.unique(y_true)) < 2:
        return float("nan")
    return average_precision_score(y_true, y_score)

def eval_probs_dnabert(y_true, probs, threshold=0.5):
    y_true = np.asarray(y_true).astype(int)
    probs = np.asarray(probs).astype(float)
    preds = (probs >= threshold).astype(int)

    return {
        "Accuracy": accuracy_score(y_true, preds),
        "Precision": precision_score(y_true, preds, zero_division=0),
        "Recall": recall_score(y_true, preds, zero_division=0),
        "F1": f1_score(y_true, preds, zero_division=0),
        "ROC_AUC": safe_auc(y_true, probs),
        "PR_AUC": safe_prauc(y_true, probs),
    }

def run_dnABERT_on_balanced_df(
    balanced_df: pd.DataFrame,
    window: int = 100,
    train_size: int = 4000,
    eval_size: int = 1000,
    epochs: int = 2,
    kmer: int = 6,
    seed: int = 42
):
    df_local = balanced_df[["sequence", "label"]].copy()
    df_local = df_local.sample(frac=1.0, random_state=seed).reset_index(drop=True)

    total_needed = train_size + eval_size
    if len(df_local) < total_needed:
        raise ValueError(f"Not enough samples in balanced_df ({len(df_local)}) for train_size+eval_size={total_needed}")

    df_train = df_local.iloc[:train_size].copy()
    df_eval  = df_local.iloc[train_size:train_size+eval_size].copy()

    df_train["sequence"] = df_train["sequence"].apply(lambda s: normalize_len(s, window))
    df_eval["sequence"]  = df_eval["sequence"].apply(lambda s: normalize_len(s, window))

    df_train["text"] = df_train["sequence"].apply(lambda s: seq_to_kmers(s, kmer))
    df_eval["text"]  = df_eval["sequence"].apply(lambda s: seq_to_kmers(s, kmer))

    train_ds = Dataset.from_pandas(df_train[["text", "label"]]).rename_column("label", "labels")
    eval_ds  = Dataset.from_pandas(df_eval[["text", "label"]]).rename_column("label", "labels")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=2)

    def tok(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

    train_ds = train_ds.map(tok, batched=True)
    eval_ds  = eval_ds.map(tok, batched=True)

    train_ds = train_ds.remove_columns(["text"])
    eval_ds  = eval_ds.remove_columns(["text"])
    train_ds.set_format("torch")
    eval_ds.set_format("torch")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
        preds = (probs >= 0.5).astype(int)

        out = {
            "accuracy": accuracy_score(labels, preds),
            "precision": precision_score(labels, preds, zero_division=0),
            "recall": recall_score(labels, preds, zero_division=0),
            "f1": f1_score(labels, preds, zero_division=0),
        }
        out["roc_auc"] = safe_auc(labels, probs)
        out["pr_auc"] = safe_prauc(labels, probs)
        return out

    args = TrainingArguments(
        output_dir="x/dnabert_out", # update this path !!!
        num_train_epochs=epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        save_strategy="no",
        logging_steps=50,
        report_to="none",
        seed=seed,
        disable_tqdm=True,
        logging_strategy="no"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    pred = trainer.predict(eval_ds)
    probs = torch.softmax(torch.tensor(pred.predictions), dim=1)[:, 1].numpy()
    labels = pred.label_ids.astype(int)

    metrics = eval_probs_dnabert(labels, probs, threshold=0.5)
    return model, tokenizer, metrics

DNABERT evaluation on UCI

In [ ]:
def dnabert_eval_on_uci(model, tokenizer, uci_df: pd.DataFrame, window: int = 100, kmer: int = 6):
    df = uci_df.copy()
    df["sequence"] = df["sequence"].apply(lambda s: normalize_len(s, window))
    texts = [seq_to_kmers(s, kmer) for s in df["sequence"].tolist()]
    y_true = df["label"].values.astype(int)

    enc = tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    enc = {k: v.to(device) for k, v in enc.items()}
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()

    return eval_probs_dnabert(y_true, probs, threshold=0.5)

Genome scanning with DNABERT

In [ ]:
def dnabert_scan_genome_fasta(model, tokenizer, fasta_path: str, window: int = 100, step: int = 10, kmer: int = 6,
                             chunk_size: int = 5000, batch_size: int = 16):
    from Bio import SeqIO
    record = SeqIO.read(fasta_path, "fasta")
    genome = clean_seq(str(record.seq))
    positions = list(range(0, len(genome) - window + 1, step))

    model = model.to(device)
    model.eval()

    all_probs = []
    for start in range(0, len(positions), chunk_size):
        chunk_pos = positions[start:start+chunk_size]
        seqs = [genome[p:p+window] for p in chunk_pos]
        texts = [seq_to_kmers(s, kmer) for s in seqs]

        enc = tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}

        chunk_probs = []
        with torch.no_grad():
            for b in range(0, len(chunk_pos), batch_size):
                batch = {k: v[b:b+batch_size] for k, v in enc.items()}
                logits = model(**batch).logits
                probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
                chunk_probs.extend(probs.tolist())

        all_probs.extend(chunk_probs)

    return positions, np.array(all_probs)

In [ ]:
dnabert_model_100, dnabert_tok_100, dnabert_A100 = run_dnABERT_on_balanced_df(
    balanced_df=balanced_df,
    window=100,
    train_size=len(balanced_df) * 0.8,
    eval_size=len(balanced_df) * 0.2,
    epochs=2
)

dnabert_model_200, dnabert_tok_200, dnabert_A200 = run_dnABERT_on_balanced_df(
    balanced_df=balanced_df,
    window=200,
    train_size=len(balanced_df) * 0.8,
    eval_size=len(balanced_df) * 0.2,
    epochs=2
)

dnabert_B100 = dnabert_eval_on_uci(dnabert_model_100, dnabert_tok_100, uci_df, window=100)
dnabert_B200 = dnabert_eval_on_uci(dnabert_model_200, dnabert_tok_200, uci_df, window=200)

# Genome scanning

DNABERT is executed locally, check out the scripts folder.

Genome scanning with BiLSTM

In [ ]:
record = SeqIO.read(FASTA_PATH, "fasta")
genome = str(record.seq).upper()
genome_len = len(genome)

# centered one-hot encoding for genome scanning
def batch_one_hot_encode_genome_centered(genome, centers, window):
    half = window // 2
    arr = np.zeros((len(centers), window, 4), dtype=np.float32)
    mapping = {'A':[1,0,0,0], 'T':[0,1,0,0], 'G':[0,0,1,0], 'C':[0,0,0,1]}

    for i, c in enumerate(centers):
        start = c - half
        end = start + window
        if start < 0 or end > len(genome):
            continue
        seq = genome[start:end]
        for j, base in enumerate(seq):
            if base in mapping:
                arr[i, j] = mapping[base]
    return arr

# chunked prediction for genome scanning
def predict_in_chunks_tf_centered(model, genome, centers, window, chunk_size=50000, batch_size=512):
    probs_all = []
    centers = list(centers)
    total = len(centers)

    for start in range(0, total, chunk_size):
        end = min(start + chunk_size, total)
        chunk_centers = centers[start:end]
        X_chunk = batch_one_hot_encode_genome_centered(genome, chunk_centers, window)
        probs_chunk = model.predict(X_chunk, batch_size=batch_size, verbose=0).ravel()
        probs_all.extend(probs_chunk)

    return np.array(probs_all), centers

# known promoter positions from RegulonDB
known_positions = promoter_df["posTSS"].dropna().astype(int).tolist()
known_positions = [p for p in known_positions if 0 <= p < genome_len]

# training BiLSTM models for genome scanning
def train_bilstm_for_scanning(window):
    Xw, yw = make_X_y_from_df(balanced_df, window)
    X_train, X_test, y_train, y_test = train_test_split(
        Xw, yw, test_size=0.2, stratify=yw, random_state=42
    )

    model = build_bilstm(window)
    es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    model.fit(X_train, y_train, validation_split=0.2, epochs=15,
              batch_size=32, callbacks=[es], verbose=0)
    return model

model_100 = train_bilstm_for_scanning(100)
model_200 = train_bilstm_for_scanning(200)


combos = [(100, 10), (100, 20), (200, 10), (200, 20)]
scan_rows = []

for window, step in combos:
    model_use = model_100 if window == 100 else model_200
    half = window // 2
    centers = range(half, genome_len - half, step)

    probs, centers_list = predict_in_chunks_tf_centered(
        model_use, genome, centers, window
    )

    thresholds = [0.4, 0.6, 0.8, 0.9] if window == 100 else [0.45]

    for threshold in thresholds:
        predicted_sites = [centers_list[i] for i, v in enumerate(probs) if v >= threshold]

        # merging nearby peaks
        merged = []
        last = -10**9
        for p in sorted(predicted_sites):
            if p - last > 200:
                merged.append(p)
                last = p

        # evaluating against known TSS (+-100 bp tolerance)
        tol = 100
        tp = sum(any(abs(k - p) <= tol for p in merged) for k in known_positions)
        fp = len(merged) - tp
        fn = len(known_positions) - tp

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        scan_rows.append({
            "Window": window,
            "Step": step,
            "Threshold": threshold,
            "MergedPeaks": len(merged),
            "TP": tp,
            "FP": fp,
            "FN": fn,
            "Precision": precision,
            "Recall": recall,
            "F1": f1
        })

df_scan = pd.DataFrame(scan_rows).sort_values(["Window", "Step", "Threshold"])
display(df_scan)